<a href="https://colab.research.google.com/github/pamunarr/Chroma/blob/main/Chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chroma - Caso de uso

**ARDA II - NoSQL**

*Andra-Iulia Cebuc, Pablo Munarriz Senosiain*

En este cuaderno mostramos un caso de uso del SGBD vectorial [Chroma](https://docs.trychroma.com/). Además de poner en práctica lo aprendido, nuestro objetivo es relacionarlo con la asignatura Aprendizaje Profundo, de forma que podamos ver el funcionamiento de Chroma en el contexto para el que fue diseñado.

## Librerías

En primer lugar, debemos instalar las librerías necesarias. Para usar Chroma basta con instalar `chromadb`, pero también utilizaremos `datasets`. Una vez instaladas, reiniciamos el kernel para vaciar la memoria RAM.

In [ ]:
!pip install chromadb
!pip install datasets -Uqq

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from datasets import load_dataset

# Dataset

Vamos a trabajar con datos de la Wikipedia, concretamente, vamos a descargarnos documentos en euskera y gallego. Para ello recurrimos a HuggingFace, en particular a [este dataset](https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3), que contiene una copia de Wikipedia del 01/11/2023 en más de 300 idiomas. Empezamos descargando los documentos en euskera.

In [ ]:
wikipedia_eu = load_dataset("Cohere/wikipedia-2023-11-embed-multilingual-v3", "eu", split="train")

Vamos a echar un vistazo al esquema de los datos.

In [3]:
wikipedia_eu

Dataset({
    features: ['_id', 'url', 'title', 'text', 'emb'],
    num_rows: 1327579
})

Tenemos cinco características:
 - `_id`: un identificador,
 - `url`: dirección URL del documento,
 - `title`: título del documento,
 - `text`: contenido del documento,
 - `emb`: un embedding del documento.

Ya que disponemos de la URL y de un embedding, vamos a aprovechar para no guardar el texto como tal, y solo trabajar con los embeddings. La propia URL nos sirve para saber a qué documento hace mención el embedding en cuestión.

Cabe mencionar que, según dicen en la página del dataset, los embeddings los han creado mediante el modelo de embedding [Cohere Embed V3](https://txt.cohere.com/introducing-embed-v3/), más concrétamente, mediante el modelo [Cohere-embed-multilingual-v3.0](https://huggingface.co/Cohere/Cohere-embed-multilingual-v3.0).

En definitiva, en nuestra colección guardaremos, para cada documento, el identificador, el embedding, el título como documento (así tendremos mayor flexibilidad en las consultas) y, como metadatos, la URL y el idioma.

Por suspuesto, antes de hablar de colecciones, debemos obtener el cliente. En este caso, trabajaremos en disco (cliente persistente), y así podremos reiniciar el kernel sin perder los datos.


In [27]:
client = chromadb.PersistentClient(path = "./chromadb")
client.heartbeat() # de esta forma comprobamos la conexión

1712250679971286188

Ahora debemos crear la colección. Para ello deberemos también conseguir la función de embedding concreta, puesto que queremos hacer consultas mediante texto y no mediante embeddings. En este caso cargaremos el modelo desde [Hugging Face](https://huggingface.co/), se puede encontrar la documentación asociada en este [enlace](https://docs.trychroma.com/embeddings/hugging-face).

In [28]:
emb_fn = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key = "" ,
    model_name = "Cohere/Cohere-embed-multilingual-v3.0"
)

collection = client.create_collection(
  name = "wikipedia" , # nombre de la colección
  embedding_function = emb_fn , # función de embedding
  metadata = {"hnsw:space" : "cosine"} # función de distancia entre embeddings
)

Ahora vamos a añadir datos a la colección. Como el dataset es bastante grande, vamos a meterlo poco a poco. Para ello primero creamos una función que cumpla con el cometido.

In [39]:
def add_dataset(ds , collection , idioma , L = 500):
  # L indica el número de documentos a añadir a la vez
  N = ds.num_rows # número total de documentos

  n_0 = 0 # posición del primer elemento a añadir
  while n_0 < N:
    n_1 = min(n_0 + L , N)

    # Nos quedamos con los documentos a añadir y escribimos también el idioma
    ds_0 = ds.select(range(n_0 , n_1)).add_column("lang" , [idioma] * (n_1 - n_0))

    # Añadimos los embeddings, junto a los metadatos y los identificadores
    collection.add(
        embeddings = ds_0["emb"] ,
        documents = ds_0["title"]
        metadatas = ds_0.select_columns(["url" , "lang"]).to_list() ,
        ids = ds_0["_id"]
    )

    n_0 += L # actualizamos n_0
    print("Porcentaje completado: %.2f" % (100 * n_0 / N))

Ya estamos listos para añadir los embeddings.

In [ ]:
add_dataset(wikipedia_eu , collection , "eu")

Comprobamos que todo está correcto. Para ello tenemos las siguientes funciones:

In [37]:
print(collection.count()) # cuenta el número de documentos
collection.peek(1) # muestra el primer elemento

1327579


{'ids': ['20231101.eu_1000001_0'],
 'embeddings': [[0.0100860595703125,
   -0.0005135536193847656,
   -0.0172271728515625,
   0.049072265625,
   -0.038818359375,
   -0.02001953125,
   0.004474639892578125,
   -2.086162567138672e-05,
   0.0095062255859375,
   -0.004390716552734375,
   -0.045562744140625,
   -0.05224609375,
   0.02874755859375,
   0.0272064208984375,
   -0.044281005859375,
   0.03363037109375,
   -0.0411376953125,
   -0.050262451171875,
   -0.01338958740234375,
   0.06976318359375,
   0.0484619140625,
   0.03521728515625,
   -0.04620361328125,
   -0.055816650390625,
   -0.01059722900390625,
   0.01226043701171875,
   0.03643798828125,
   -0.01262664794921875,
   -0.0008854866027832031,
   0.0176544189453125,
   -0.044952392578125,
   0.005466461181640625,
   0.01358795166015625,
   0.0067291259765625,
   -0.0151214599609375,
   -0.0247039794921875,
   0.0039520263671875,
   -0.05035400390625,
   -0.0186614990234375,
   0.03857421875,
   0.0211181640625,
   -0.01094055175

Ya tenemos los documentos en euskara en la base de datos, ahora vamos a hacer lo mismo para los documentos en gallego.

Vamos a aprovechar a reiniciar el kernel, así que tendremos que reescribir el código ya que perderemos las variables.

In [2]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from datasets import load_dataset

In [ ]:
wikipedia_gl = load_dataset("Cohere/wikipedia-2023-11-embed-multilingual-v3", "gl", split="train").remove_columns("text")

In [ ]:
client = chromadb.PersistentClient(path = "./chromadb")

emb_fn = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key = "" ,
    model_name = "Cohere/Cohere-embed-multilingual-v3.0"
)

collection = client.get_collection(
  name = "wikipedia" ,
  embedding_function = emb_fn ,
)

def add_dataset(ds , collection , idioma , L = 500):
  # L indica el número de documentos a añadir a la vez
  N = ds.num_rows # número total de documentos

  n_0 = 0 # posición del primer elemento a añadir
  while n_0 < N:
    n_1 = min(n_0 + L , N)

    # Nos quedamos con los documentos a añadir y escribimos también el idioma
    ds_0 = ds.select(range(n_0 , n_1)).add_column("lang" , [idioma] * (n_1 - n_0))

    # Añadimos los embeddings, junto a los metadatos y los identificadores
    collection.add(
        embeddings = ds_0["emb"] ,
        documents = ds_0["title"]
        metadatas = ds_0.select_columns(["url" , "lang"]).to_list() ,
        ids = ds_0["_id"]
    )

    n_0 += L # actualizamos n_0
    print("Porcentaje completado: %.2f" % (100 * n_0 / N))

add_dataset(wikipedia_gl , collection , "gl")

In [4]:
collection.count()

1849579

## Consultas

Ya hemos terminado de cargar documentos. Ahora nos interesa consultar la colección. Trataremos de mostrar las distintas herramientas que tenemos a nuestra disposición.

En primer lugar, vamos simplemente a filtrar embeddings como una base de datos al uso. Por ejemplo, vamos a quedarnos con los URL de los documentos en euskera cuyo título contenga la palabra "Lizarra". Para esto tenemos el método `get`.

In [3]:
client = chromadb.PersistentClient(path = "./chromadb")

emb_fn = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key = "" ,
    model_name = "Cohere/Cohere-embed-multilingual-v3.0"
)

collection = client.get_collection(
  name = "wikipedia" ,
  embedding_function = emb_fn ,
)

In [ ]:
collection.get(
    include = ["metadatas"] ,
    where = {
        "lang" : "eu"
    } ,
    where_document = {"$contains" : "Lizarra"}
)